In [ ]:
from ambry import get_library
import pandas as pd
l = get_library()

geofile_schema_p = l.partition('census.gov-acs_geofile-schemas-2009e-geofile_schema-2014')
col_meta_p = l.partition('census.gov-acs_geofile-schemas-2009e-column_meta-2014-5')
table_meta_p = l.partition('census.gov-acs_geofile-schemas-2009e-table_meta-2014-5')
seq_p = l.partition('census.gov-acs_geofile-schemas-2009e-table_sequence-2014-5')
shell_p = l.partition('census.gov-acs_geofile-schemas-2009e-table_shells-2014-5')

In [ ]:
shell_p.table

In [ ]:
shell_p.analysis.dataframe().head()

In [ ]:
table_meta_p.table

In [ ]:
table_meta_p.analysis.dataframe().head()

In [ ]:
col_meta_p.table

In [ ]:
col_meta_p.analysis.dataframe().head()

In [ ]:
from collections import defaultdict
from itertools import islice
header = 'Section Schema Datatype Title Indent Subject Universe Topics Denominator Parent'.split()
rows = []
tables = []


def mk_table_row(row):
    return {
        'Section': 'Table',
        'Schema': row['table_id'],
        'Title': row['table_title'],
        'Subject': row['subject_area'],
        'Universe': row['universe'],
        'Topics': row['topics'],
        'Denominator': row['denominator_column_id']
    }

for t in table_meta_p:
    tables.append(mk_table_row(t.dict))

print tables[0]

In [ ]:
from collections import defaultdict
columns = defaultdict(list)

def mk_column_row(row):

    return {
        'Section': 'Column',
        'Schema': row['column_id'],
        'Title': row['column_title'],
        'Indent': row['indent'],
        'Parent': row['parent_column_id']
    }
    
for c in col_meta_p:
    columns[c.table_id].append(mk_column_row(c.dict))

In [ ]:
from unicodecsv import DictWriter
header = 'Section Schema Datatype Title Indent Subject Universe Topics Denominator Parent'.split()

with open('/tmp/foo.csv', 'w') as f:
    w = DictWriter(f, header)
    w.writeheader()

    for t in tables:
        w.writerow(t)
        for c in columns[t['Schema']]:
            w.writerow(c)

In [65]:
import unicodecsv as csv

b = l.bundle('census.gov-acs-p5ye2014')
large_url_template = b.source('large_area_url').ref
small_url_template = b.source('small_area_url').ref
url_root = b.source('base_url').ref

def states():
    """Return tuples of states, which can be used to make maps and lists"""

    states = set()
    with b.dep('states').reader as r:
        for row in r.select( lambda r: r['component'] == '00'):
            if row['stusab'].upper() == 'DC':
                states.add((row['stusab'], row['state'], 'DistrictOfColumbia')) # Orig lowercases 'Of'
            else:
                states.add((row['stusab'], row['state'], row['name'] ))
    return list(states)

with open('/tmp/data.csv','w') as f:
    w = csv.writer(f)

    for stusab, state_id, state_name in states():

        for grain, tmpl in (('l',large_url_template), ('s',small_url_template)):
            url = tmpl.format(root=url_root, state_name=state_name).replace(' ', '')

            w.writerow(['Datafile',url, stusab, grain])